In [2]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [3]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [4]:
# Get the list of supported facilities
org_unit_group = 'gH2DlwAo1ja'
org_units = api.get(f'organisationUnitGroups/{org_unit_group}', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [5]:
# period of analysis
period = '2024Q4'
semmiannual = '2024AprilS1'
annual = '2023Oct'
mecanism = 81002

In [5]:
# QUARTERLY DATA
quarterly_data_element_group = 'DJ8usz0OCUB'
data_values = api.get('dataValueSets', params ={'fields':'dataElement, period, orgUnit, categoryOptionCombo, attributeOptionCombo, value', 'period':period, 'orgUnitGroup':org_unit_group, 'dataElementGroup':quarterly_data_element_group})
data_values = data_values.json()['dataValues']

In [6]:
# SEMIANNUAL DATA
semmiannual_data_element_group = 'eEP8HzyCIUZ'
semmiannual_data = api.get('dataValueSets', params ={'fields':'dataElement, period, orgUnit, categoryOptionCombo, attributeOptionCombo, value', 'period':semmiannual, 'orgUnitGroup':org_unit_group, 'dataElementGroup':semmiannual_data_element_group})
semmiannual_data = semmiannual_data.json()['dataValues']

In [15]:
# ANNUAL DATA
annual_data_element_group = 'OymNguXaXyG'
annual_data = api.get('dataValueSets', params ={'fields':'dataElement, period, orgUnit, categoryOptionCombo, attributeOptionCombo, value', 'period':annual, 'orgUnitGroup':org_unit_group, 'dataElementGroup':annual_data_element_group})
annual_data = annual_data.json()['dataValues']

In [8]:
data_values = data_values + semmiannual_data + annual_data

In [9]:
# quarterly
data_elements = api.get(f'dataElementGroups/{quarterly_data_element_group}', params={'fields':'dataElements'})
data_elements = data_elements.json()['dataElements']

# # semi-annual
# semmiannual_data_elements = api.get(f'dataElementGroups/{semmiannual_data_element_group}', params={'fields':'dataElements'})
# semmiannual_data_elements = semmiannual_data_elements.json()['dataElements']

# # annual
# annual_data_elements = api.get(f'dataElementGroups/{annual_data_element_group}', params={'fields':'dataElements'})
# annual_data_elements = annual_data_elements.json()['dataElements']

# data_elements = data_elements + semmiannual_data_elements + annual_data_elements

In [10]:
for data_element in data_elements:

    data_element_id = data_element['id']

    indicators_metadata = api.get('dataElementOperands', params={'paging':False, 'fields':'id,name', 'filter':f'dataElement.id:eq:{data_element_id}'})
    indicators_metadata = indicators_metadata.json()['dataElementOperands']

    for metadata_indicator in indicators_metadata:

        for data_value in data_values:
            
            data_value_id = data_value['dataElement']+'.'+ data_value['categoryOptionCombo']
            indicator_id = metadata_indicator['id']
            indicator_name = metadata_indicator['name']

            if data_value_id == indicator_id:
                name = indicator_name
                name = name.replace('MER_', '')
                name = name.replace('MER25_', '')
                name = name.replace('Known at Entry Positive', 'Known Positives')
                name = name.replace('Newly Identified Positive', 'Newly Tested Positives')
                name = name.replace('Newly Identified Negative', 'New Negatives')
                name = name.replace('PMTCT_ART (N, TA, Age/NewExistingArt/Sex/HIVStatus)', 'PMTCT_ART (N, TA, Age/Sex/NewExistingArt/HIVStatus)')
                name = name.replace('PMTCT_HEI_POS_ART (N, TA, Age/HIVStatus/ARTStatus): Infant Testing 2 - 12 months, Positive, Receiving ART', 'PMTCT_HEI_POS_ART (N, TA, Age/HIVStatus/ARTStatus): Infant Testing 2 - 12 months , Positive, Receiving ART')

                # TX_RTT_IIT
                if data_value['dataElement'] == 'euQty0yYWEk':
                    name = name.replace('Months Treatment', 'Months Interruption')

                # TX_RTT
                if data_value['dataElement'] == 'EXHtjo00lEh':
                    name_main = 'TX_RTT (N, TA, Age/Sex/CD4/HIVStatus): Restarted ARV '
                    name_description = name.split(name_main)[1]
                    name_description = name_description.split(', ')[3]+', '+name_description.split(', ')[0]+', '+name_description.split(', ')[1]+', '+name_description.split(', ')[2]
                    name = name_main+name_description

                # TX_NEW
                if data_value['dataElement'] == 'qXkiKCAX5yX':
                    name_main = 'TX_NEW (N, TA, Age/Sex/CD4/HIVStatus): New on ART '
                    name_description = name.split(name_main)[1]
                    name_description = name_description.split(', ')[3]+', '+name_description.split(', ')[0]+', '+name_description.split(', ')[1]+', '+name_description.split(', ')[2]
                    name = name_main+name_description
                
                # TX_ML
                if data_value['dataElement'] == 'hvt2KcpoLMp':
                    name = name.replace('No Contact Outcome - Interruption In Treatment (6+ Months Treatment), Positive', 'Positive, No Contact Outcome - Interruption In Treatment (6+ Months Treatment)')
                    name_main = 'TX_ML (N, TA, Age/Sex/ARTNoContactReason/HIVStatus): On ART no clinical contact '
                    name_description = name.split(name_main)[1]

                    if name_description.split(', ')[3] == 'Positive':
                        name_description = name_description.split(', ')[0]+', '+name_description.split(', ')[1]+', '+name_description.split(', ')[3]+', '+name_description.split(', ')[2]
                        name = name_main+name_description

                # TX_PVLS
                if data_value['dataElement'] == 'X20YBFt9lit':
                     name = name.replace('Viral Load Documented', 'Viral Load Documented Female,')

                if data_value['dataElement'] == 'M4jFFs2xTAx':
                     name = name.replace('12 Months Viral Load < 1000', '12 Months Viral Load < 1000 Female,')

                data_value['data_value_name'] = name

                # Fix facilities
                if data_value['orgUnit'] == 'U3j0Fn86BVV':
                   data_value['orgUnit'] = 'uonm9XithMw'

                if data_value['orgUnit'] == 'ZYfAiZnlNsQ':
                   data_value['orgUnit'] = 'GGEtuWJUhLH'

                if data_value['orgUnit'] == 'dtAHOODVMmB':
                   data_value['orgUnit'] = 'zLJdhkaPocX'

                if data_value['orgUnit'] == 'KYB11OcHLUo':
                   data_value['orgUnit'] = 'QFPMqEHV52d'

                if data_value['orgUnit'] == 'oprOfxPSkim':
                   data_value['orgUnit'] = 'Tp2GlPj1Zp9'

                if data_value['orgUnit'] == 'K4Rp81pn9ke':
                   data_value['orgUnit'] = 'RUQjZiHdXwL'

In [11]:
saurino_atip_data = {}

for data_value in data_values:

    if data_value['orgUnit'] == 'RUQjZiHdXwL':

        indicator_key = data_value['dataElement']+ '.'+data_value['categoryOptionCombo']

        if indicator_key not in saurino_atip_data:
           saurino_atip_data[indicator_key] = data_value
        else:
           saurino_atip_data[indicator_key]['value'] = int(saurino_atip_data[indicator_key]['value']) + int(data_value['value'])

saurino_atip_data = list(saurino_atip_data.values())

In [12]:
 # remove old data
new_data_values = []

for data_value in data_values:
    
    if data_value['orgUnit'] != 'RUQjZiHdXwL':
        new_data_values.append(data_value)

data_values = new_data_values
data_values = data_values + saurino_atip_data

In [13]:
len(data_values)

6837

In [14]:
data_values = pd.json_normalize(data_values)

datim_data = pd.read_csv('DATIM_DATA_ELEMENTS.csv')
datim_data = datim_data[['dataelement', 'categoryoptioncombo', 'categoryoptioncombocode']]
datim_data['name'] = datim_data['dataelement']+' '+datim_data['categoryoptioncombo']
data_values = data_values.merge(datim_data, left_on='data_value_name', right_on='name', how='left')
data_values['categoryOptionCombo'] = data_values['categoryoptioncombocode']
data_values['attributeOptionCombo'] = mecanism
data_values['period'] = period
data_values = data_values[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]

data_values.to_csv('MER_DATA.csv', index=False)